# Aufgabe 11.3

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Sep 21 13:50:37 2021

@author: Chantal
"""
import os
import pandas as pd
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS

#Festsetzen des Pfads zum Odner, in dem sich das Scriptfile und der Datensatz befinden
#os.chdir("C:/Users/Chantal/sciebo/Ewifo/WS2122/Übung_neu/Thema 11")

# Auslesen der Daten
data = pd.read_excel("cigarettes1.xlsx","Sheet1")
print('Variablen im Datensatz: \n', data.columns.values)

# Abhängige Variablen und Regressoren
y=data[['logQcig']]
x1=data[['logPcig']]
x = pd.DataFrame(sm.add_constant(x1.values, has_constant='add'), \
                    columns = ['constant'] + x1.columns.tolist())

# Schätzung ohne Berücksichtigung der Endogenität
cigreg = sm.OLS(y,x).fit()

Variablen im Datensatz: 
 ['Tcig' 'logQcig' 'logPcig']


# a)


In [2]:
# Schätzung der ersten Stufe der TSLS-Regression. Annahme: ln(Pcig) ist die endogene Variable
# und Tcig das Instrument. In der 1. Stufe der TSLS-Schätzung ist die endogene Variable
# ln(Pcig) die abhängige Variable und das Instrument der Regressor.
x_IV = data[['Tcig']]
y_IV = x1 #logpcig
x_IV = pd.DataFrame(sm.add_constant(x_IV.values, has_constant='add'), columns = ['Konstante'] + x_IV.columns.tolist())
ivreg1 = sm.OLS(y_IV,x_IV).fit()
print('Koeffizienten der 1. Stufe der TSLS-Schätzung \n', ivreg1.params)

Koeffizienten der 1. Stufe der TSLS-Schätzung 
 Konstante    4.788914
Tcig         0.007707
dtype: float64


In [5]:
ivreg1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                logPcig   R-squared:                       0.895
Model:                            OLS   Adj. R-squared:                  0.893
Method:                 Least Squares   F-statistic:                     391.4
Date:                Sat, 19 Mar 2022   Prob (F-statistic):           3.94e-24
Time:                        20:25:07   Log-Likelihood:                 85.204
No. Observations:                  48   AIC:                            -166.4
Df Residuals:                      46   BIC:                            -162.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Konstante      4.7889      0.022    219.944      0.000       4.745       4.833
Tcig           0.0077      0.000     19.783      0.000       0.007       0.008
==============================================================================
Omnibus:                        5.893   Durbin-Watson:                   1.767
Prob(Omnibus):                  0.053   Jarque-Bera (JB):                4.786
Skew:                           0.726   Prob(JB):                       0.0913
Kurtosis:                       3.533   Cond. No.                         201.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [3]:
# Schätzung der zweiten Stufe der TSLS-Regression. In der 2. Stufe der TSLS-Schätzung ist
# ln(Qcig) die ahängige Variable und die geschätzte Variable hat(ln(Pcig)) der 1. Stufe
# ist der Regressor.
data['Pcighat']=ivreg1.predict(x_IV)
x_IV2= data[['Pcighat']]
x_IV2 = pd.DataFrame(sm.add_constant(x_IV2.values, has_constant='add'), columns = ['Konstante'] + x_IV2.columns.tolist())
y_IV2= y #Qcig
ivreg2 = sm.OLS(y_IV2, x_IV2).fit()
print('Koeffizienten der 2. Stufe der TSLS-Schätzung \n', ivreg2.params)

Koeffizienten der 2. Stufe der TSLS-Schätzung 
 Konstante    10.523135
Pcighat      -1.150225
dtype: float64


In [4]:
ivreg2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                logQcig   R-squared:                       0.326
Model:                            OLS   Adj. R-squared:                  0.312
Method:                 Least Squares   F-statistic:                     22.29
Date:                Sat, 19 Mar 2022   Prob (F-statistic):           2.22e-05
Time:                        20:22:28   Log-Likelihood:                 9.7174
No. Observations:                  48   AIC:                            -15.43
Df Residuals:                      46   BIC:                            -11.69
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Konstante     10.5231      1.268      8.301      0.000       7.971      13.075
Pcighat       -1.1502      0.244     -4.722      0.000      -1.641      -0.660
==============================================================================
Omnibus:                       10.662   Durbin-Watson:                   1.809
Prob(Omnibus):                  0.005   Jarque-Bera (JB):               13.053
Skew:                          -0.746   Prob(JB):                      0.00146
Kurtosis:                       5.074   Cond. No.                         235.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""


# c)


In [6]:
print('Steigungskoeffizient der 2.Stufe der TSLS-Schätzung: \n ', round(ivreg2.params['Pcighat'],4))

Steigungskoeffizient der 2.Stufe der TSLS-Schätzung: 
  -1.1502



# e)


In [7]:
resultIV = IV2SLS(y,x,x_IV).fit()
print('Koeffizienten der IV-Schätzung mit richtigen Standardfehlern \n', resultIV.params)

print('Standardfehler der 2. Stufe der TSLS-Schätzung \n', ivreg2.bse)
print('Standardfehler der IV-Schätzung mit richtigen Standardfehlern \n', resultIV.bse)

Koeffizienten der IV-Schätzung mit richtigen Standardfehlern 
 constant    10.523135
logPcig     -1.150225
dtype: float64
Standardfehler der 2. Stufe der TSLS-Schätzung 
 Konstante    1.267760
Pcighat      0.243608
dtype: float64
Standardfehler der IV-Schätzung mit richtigen Standardfehlern 
 constant    1.191873
logPcig     0.229026
dtype: float64
